## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [48]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice


def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [53]:
def sp_tree(matrix):
    n = len(matrix)
    used = [0 for i in range(n)]
    parent = [0 for i in range(n)]
    dist = matrix[0][:]
    used[0] = 1
    parent[0] = -1
    ans = 0
    for i in range(n-1):
        best = -1
        bestdist = 0
        for v in range(n):
            if not used[v]:
                if best == -1 or bestdist > dist[v]:
                    best = v
                    bestdist = dist[v]
        v = best
        ans += dist[v]
        used[v] = 1
        for w in range(n):
            if not used[w]:
                if dist[w] > matrix[v][w]:
                    dist[w] = matrix[v][w]
                    parent[w] = v

    return (parent, ans)

In [343]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    n = len(vertex_coordinates)
    matrix = [[euclidean_distance(a, b) for a in vertex_coordinates] for b in vertex_coordinates]
    a, opt = sp_tree(matrix)
    y = [0 for i in range(n)]
    upper = 2 * opt
    
    iters = int(1e7 / (n**2)) + 2
    maxchanges = 10
    t = 100
    eps = 1e-3
    alpha = 2
    beta = 0.95
    if iters < 10:
        alpha = 0.1
        beta = 0.5
    curedges = [[matrix[i][j] for i in range(n)] for j in range(n)]
    for iteration in range(iters):
        tree = sp_tree(curedges)
        parent, bound = tree
        bound += 2 * sum(y)
        opt = max(bound, opt)
        deg = [0 for i in range(n)]
        summ = 0
        for v in range(n):
            if parent[v] != -1:
                deg[v] += 1
                deg[parent[v]] += 1
        for v in range(n):
            summ += ((2 - deg[v]) ** 2)

        t = (alpha * (upper - bound)) / summ
        if t < eps:
            break
        for v in range(n):
            y[v] += t * (2 - deg[v])
        for u in range(n):
            for v in range(n):
                curedges[u][v] = matrix[u][v] - y[u] - y[v]
        alpha *= beta
    return opt


In [344]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [345]:
v = [(0, 0), (1, 2), (3, 4), (2, 2)]
print(lower_bound_tsp(v))
print(1+2*sqrt(5))
print(5 + 2 * sqrt(5))

12.26919426949878
5.47213595499958
9.47213595499958


In [346]:
run_all()

Instance pr107.tsp… done in 2.3 seconds with lower bound 38528
Instance pr152.tsp… done in 5.2 seconds with lower bound 65672
Instance d198.tsp… done in 6.8 seconds with lower bound 14172
Instance pr439.tsp… done in 7.0 seconds with lower bound 100531
Instance d493.tsp… done in 7.2 seconds with lower bound 31813
Instance d657.tsp… done in 7.6 seconds with lower bound 42490
Instance d2103.tsp… done in 1.8e+01 seconds with lower bound 76300


## Выводы
Запишите здесь полученные результаты относительно погрешностей алгоритмов NN и NI.

Пусть x - найденное каким-то алгоритмом значение цикла, а y - ценка снизу на значение. Тогда y <= opt <= x. Тогда погрешность = (x - opt) / opt <= (x - y) / y

d198.tsp… done in 0.037 seconds with tour length 18570 using NN and in 0.11 seconds with tour length 18052 using NI
Instance d198.tsp… done in 6.8 seconds with lower bound 14172

Погрешность NN: (18570 - 14172) / 14172 = 31%, NI: 27%

Solving instance d493.tsp… done in 0.14 seconds with tour length 43630 using NN and in 0.61 seconds with tour length 41576 using NI
Instance d493.tsp… done in 7.2 seconds with lower bound 31813

Погрешность NN: (43630 - 31813) / 31813 = 37%, NI: 30%

Solving instance d657.tsp… done in 0.25 seconds with tour length 62076 using NN and in 1.1 seconds with tour length 60195 using NI
Instance d657.tsp… done in 7.6 seconds with lower bound 42490

Погрешность NN: (62076 - 42490) / 42490 = 46%, NI: 41%

Solving instance d2103.tsp… done in 2.6 seconds with tour length 87323 using NN and in 1.1e+01 seconds with tour length 86027 using NI



Solving instance pr107.tsp… done in 0.0064 seconds with tour length 46480 using NN and in 0.027 seconds with tour length 53211 using NI
Solving instance pr152.tsp… done in 0.012 seconds with tour length 85567 using NN and in 0.054 seconds with tour length 86914 using NI
Solving instance pr439.tsp… done in 0.1 seconds with tour length 131056 using NN and in 0.46 seconds with tour length 132780 using NI